# Attack Code Testing

## Imports

In [1]:
from adversary import *
from net import Net
import numpy as np
from torch.optim import Optimizer
import torch
from training import training, testing, accuracy
from optimizer import MiniBatchOptimizer
import matplotlib.pyplot as plt
from data_utils import get_mnist, build_data_loaders

## Setup of a Testing Net with Minibatch

In [2]:
accuracies_naive_fgsm = []
accuracies_naive_proj = []

criterion = torch.nn.CrossEntropyLoss()

epochs = 10
batch_size = 100
learning_rate = 0.01
decreasing_lr = False
use_cuda = True # GPU seems to raise erros on my side
device = torch.device('cuda' if use_cuda and torch.cuda.is_available() else 'cpu')


net_naive = Net().to(device)
train_dataset, test_dataset = get_mnist(normalize=True)
train_loader, test_loader = build_data_loaders(train_dataset, test_dataset, batch_size)

In [3]:
epsilons = np.arange(0, 0.5, 0.05)

## Train Naive Model

In [4]:
mini_opt = MiniBatchOptimizer(net_naive.parameters(), lr=learning_rate, decreasing_lr=decreasing_lr)
losses, acc = training(net_naive, train_loader, mini_opt, criterion, accuracy, epochs=epochs, device=device)
losses, acc = testing(net_naive, test_loader, criterion, accuracy, device=device)


Launching training on cuda batch 100	loss = 1.666	acc = 0.73
batch 200	loss = 0.6291	acc = 0.82
batch 300	loss = 0.6094	acc = 0.81
batch 400	loss = 0.3901	acc = 0.92
batch 500	loss = 0.2852	acc = 0.93
batch 600	loss = 0.3494	acc = 0.92
epoch 0	avg epoch loss = 0.7691	avg epoch acc = 0.8143
batch 100	loss = 0.1683	acc = 0.97
batch 200	loss = 0.1808	acc = 0.95
batch 300	loss = 0.3013	acc = 0.87
batch 400	loss = 0.2257	acc = 0.92
batch 500	loss = 0.1399	acc = 0.96
batch 600	loss = 0.2965	acc = 0.96
epoch 1	avg epoch loss = 0.2069	avg epoch acc = 0.9389
batch 100	loss = 0.1007	acc = 0.97
batch 200	loss = 0.1095	acc = 0.97
batch 300	loss = 0.1785	acc = 0.94
batch 400	loss = 0.1427	acc = 0.96
batch 500	loss = 0.08739	acc = 0.98
batch 600	loss = 0.2777	acc = 0.97
epoch 2	avg epoch loss = 0.1416	avg epoch acc = 0.9588
batch 100	loss = 0.06902	acc = 0.98
batch 200	loss = 0.07592	acc = 0.97
batch 300	loss = 0.1175	acc = 0.97
batch 400	loss = 0.1002	acc = 0.96
batch 500	loss = 0.06249	acc = 1.0
b

## Attack Naive Model

Standard FGSM

In [8]:
losses, acc = attack(net_naive, criterion, accuracy, test_loader, epsilon=0.0, device=device)
print("Naive Net: ", acc)

Epsilon: 0.00	Test Accuracy = 0.900
Naive Net:  0.9004999999999999


In [ ]:
for eps in epsilons:
    loss_attack, acc_attack  = attack(net_naive, criterion, accuracy, test_loader, epsilon=eps, device=device)
    accuracies_naive_fgsm.append(acc_attack)

Projected Gradient Descent

In [ ]:
epsilons_proj

In [9]:
for eps in epsilons:
    loss_attack, acc_attack  = projected_attack(net_naive, criterion, accuracy, test_loader, epsilon=eps, alpha=0.1, num_iter=40, device=device)
    accuracies_naive_proj.append(acc_attack)


Epsilon: 0.00	Test Accuracy = 0.983
Epsilon: 0.05	Test Accuracy = 0.962
Epsilon: 0.10	Test Accuracy = 0.919
Epsilon: 0.15	Test Accuracy = 0.835
Epsilon: 0.20	Test Accuracy = 0.692
Epsilon: 0.25	Test Accuracy = 0.498
Epsilon: 0.30	Test Accuracy = 0.302
Epsilon: 0.35	Test Accuracy = 0.172
Epsilon: 0.40	Test Accuracy = 0.094
Epsilon: 0.45	Test Accuracy = 0.044


## Train Robust Models Version

In [10]:
robust_net = Net().to(device)
protect_epochs = 10
protect_lr = 0.01
protect_bz = 32
protect_dec_lr = False

prot_train_loader, prot_test_loader = build_data_loaders(train_dataset, test_dataset, protect_bz)
mini_opt_proc = MiniBatchOptimizer(robust_net.parameters(), lr=protect_lr, decreasing_lr=protect_dec_lr)

accuracies_robust_fgsm = []
accuracies_robust_proj = []

In [11]:
loss_train, acc_train = protected_training(robust_net, prot_train_loader, mini_opt_proc, criterion, accuracy, epochs=protect_epochs, device=device)
loss_test, acc_test = testing(robust_net, test_loader, criterion, accuracy, device=device)

KeyboardInterrupt: 

## Attack Robust Model Version

Standard FGSM

In [ ]:
for eps in epsilons_fgsm:
    loss_attack, acc_attack  = attack(robust_net, criterion, accuracy, test_loader, epsilon=eps, device=device)
    accuracies_robust_fgsm.append(acc_attack)

Projected Gradient Descent

In [ ]:
for eps in epsilons_proj:
    loss_attack, acc_attack  = projected_attack(robust_net, criterion, accuracy, test_loader, epsilon=eps, alpha=1e-2, num_iter=40, device=device)
    accuracies_robust_proj.append(acc_attack)


## Comparative Analysis

In [ ]:
epsilons_it = np.arange(0, len(epsilons_pgd), 1)


### Naive Model

In [ ]:
plt.figure(figsize=(5,5))
plt.plot(epsilons_it, accuracies_naive_fgsm, "*-", c='blue', label='Std. FGSM')
plt.plot(epsilons_it, accuracies_naive_proj, "*-", c='orange', label='PGD')

plt.yticks(np.arange(0, 1.1, step=0.1))
plt.xticks(epsilons_it)

plt.title("Naive Model")
plt.xlabel("Epsilon Choices")
plt.ylabel("Accuracy")
plt.legend();

### Robust Mode

In [ ]:
plt.figure(figsize=(5,5))
plt.plot(epsilons_it, accuracies_robust_fgsm, "*-", c='blue', label='Std. FGSM')
plt.plot(epsilons_it, accuracies_robust_proj, "*-", c='orange', label='PGD')

plt.yticks(np.arange(0, 1.1, step=0.1))
plt.xticks(epsilons_it)

plt.title("Robust Model")
plt.xlabel("Epsilon Choices")
plt.ylabel("Accuracy")
plt.legend();

## File output

In [ ]:
print("********************************")
for entry in  accuracies_naive_fgsm:
    print(entry)
print("********************************")
for entry in  accuracies_naive_proj:
    print(entry)
print("********************************")
for entry in  accuracies_robust_fgsm:
    print(entry)
print("********************************")
for entry in  accuracies_robust_proj:
    print(entry)